In [2]:
import numpy as np
from cs231n import optim
import matplotlib.pyplot as plt
import tensorflow as tf

from cs231n.layers import *
from cs231n.rnn_layers import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


# Parameters for the model and dataset.
TRAINING_SIZE = 10000
DIGITS = 7
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 2

# All the numbers, plus sign and space for padding.
chars = '0123456789se '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    # generate data one less than specified digit length so that we can have input and output of same size
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS))))
    a = f()
    b = a+1
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = (a, b)
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = 's{}e'.format(a)
    query = ' ' * (MAXLEN - len(q)) + q
    ans = 's{}e'.format(b)
    # Answers can be of maximum size DIGITS + 1.
    ans = ' ' * (MAXLEN - len(ans)) + ans
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        ans = ans[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.int32)
y = np.zeros((len(questions), MAXLEN ), dtype=np.int32)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = np.array([ctable.char_indices[z] for z in sentence])

    
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
# because it is sorted
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
('Total addition questions:', 10000)
Vectorization...
Training Data:
(9000, 9, 13)
(9000, 9)
Validation Data:
(1000, 9, 13)
(1000, 9)


In [ ]:
print (x_train[0], y_train[0])

In [4]:
D = len(chars)
C = len(chars)
T = MAXLEN
N = 5
H = 150
num_epochs = 5
num_batches = x_train.shape[0]//N
batch_size = N

In [5]:
tf.reset_default_graph()

batchX_placeholder = tf.placeholder(tf.float32, [None,T,D], name='x_batch')
batchY_placeholder = tf.placeholder(tf.int32, [None,T], name='y_batch')

N = tf.shape(batchX_placeholder)[0]
cell_state = tf.zeros([N,H], dtype=tf.float32)
hidden_state = tf.zeros([N,H], dtype=tf.float32)
init_state = tf.contrib.rnn.LSTMStateTuple(cell_state, hidden_state)

W2 = tf.Variable(np.random.rand(H,C),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,C)), dtype=tf.float32)

# Forward passes
with tf.variable_scope('rnn1'):
    cell = tf.contrib.rnn.BasicLSTMCell(H, state_is_tuple=True)
    states_series, current_state = tf.nn.dynamic_rnn(cell, batchX_placeholder, initial_state=init_state)

losses = []
predictions = []
with tf.variable_scope('rnn2'):
    cell = tf.contrib.rnn.BasicLSTMCell(H, state_is_tuple=True, reuse=tf.get_variable_scope().reuse)
    current_input = tf.zeros([N,D], dtype=tf.float32)
    current_input += (ctable.encode('s',1)).astype(np.float32)
    for t in range(T):
        if t > 0: tf.get_variable_scope().reuse_variables()
        states_series, current_state = cell(tf.stack(current_input), current_state)
        logits = tf.matmul(states_series, W2) + b2 #Broadcasted addition
        labels = tf.reshape(batchY_placeholder[:,t], [-1])
        losses.append(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
        prob = tf.nn.softmax(logits)
        pred = tf.one_hot(tf.argmax(prob, axis=1), C, axis=-1)
        current_input = pred
        predictions.append(pred)

predictions = tf.stack(predictions, axis=1, name='pred')
print (predictions.shape)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

(?, 9, 13)


In [5]:
saver = tf.train.Saver()
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        sess.run(tf.global_variables_initializer())
        loss_list = []

        for epoch_idx in range(num_epochs):
            print("New data, epoch", epoch_idx)

            for batch_idx in range(num_batches):
                start_idx = batch_idx * batch_size
                end_idx = start_idx + batch_size

                batchX = x_train[start_idx:end_idx]
                batchY = y_train[start_idx:end_idx]

                _total_loss, _train_step, batchP = sess.run(
                    [total_loss, train_step, predictions],
                    feed_dict={
                        batchX_placeholder: batchX,
                        batchY_placeholder: batchY
                    })

                loss_list.append(_total_loss)

                if batch_idx%100 == 0:
                    print("Step",batch_idx, "Batch loss", _total_loss)
                    #plot(loss_list, _predictions_series, batchX, batchY)

            batchX = x_val
            batchY = y_val

            batchP = sess.run(
                predictions,
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY
                })
            w = [''.join(ctable.indices_char[i] for i in batchY[j]) for j in range(x_val.shape[0])]
            q = [ctable.decode(batchP[i]) for i in range(x_val.shape[0])]
            correct_examples = 0
            for i,j in zip(w,q):
                if (i==j):
                    correct_examples += 1
            print ('Validation accuracy', float(correct_examples)/x_val.shape[0])

            batchX = x_train
            batchY = y_train

            batchP = sess.run(
                predictions,
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY
                })
            w = [''.join(ctable.indices_char[i] for i in batchY[j]) for j in range(x_train.shape[0])]
            q = [ctable.decode(batchP[i]) for i in range(x_train.shape[0])]
            correct_examples = 0
            for i,j in zip(w,q):
                if (i==j):
                    correct_examples += 1
            print ('Training accuracy', float(correct_examples)/x_train.shape[0])

        saver.save(sess, 'my-model')

('New data, epoch', 0)
('Step', 0, 'Batch loss', 2.5285707)
('Step', 100, 'Batch loss', 1.7185457)
('Step', 200, 'Batch loss', 1.4127274)
('Step', 300, 'Batch loss', 1.2988453)
('Step', 400, 'Batch loss', 1.0313483)
('Step', 500, 'Batch loss', 0.97316241)
('Step', 600, 'Batch loss', 0.94167984)
('Step', 700, 'Batch loss', 0.65194541)
('Step', 800, 'Batch loss', 0.69190645)
('Step', 900, 'Batch loss', 0.61823708)
('Step', 1000, 'Batch loss', 1.820441)
('Step', 1100, 'Batch loss', 0.71741045)
('Step', 1200, 'Batch loss', 0.53464592)
('Step', 1300, 'Batch loss', 0.66126275)
('Step', 1400, 'Batch loss', 0.57788455)
('Step', 1500, 'Batch loss', 0.39050192)
('Step', 1600, 'Batch loss', 0.41159782)
('Step', 1700, 'Batch loss', 0.49800003)
('Validation accuracy', 0.274)
('Training accuracy', 0.2897777777777778)
('New data, epoch', 1)
('Step', 0, 'Batch loss', 0.46082979)
('Step', 100, 'Batch loss', 0.36494273)
('Step', 200, 'Batch loss', 0.3436076)
('Step', 300, 'Batch loss', 0.24188267)
('Ste

In [8]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('my-model.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))
    
    q = '      s1e'
    a = 1
    batchX = np.reshape(ctable.encode(q, MAXLEN),(1,MAXLEN,C))
    
    graph = tf.get_default_graph()
    x_batch = graph.get_tensor_by_name("x_batch:0")
    pred = graph.get_tensor_by_name("pred:0")
    
    for i in range(1000):
        if i%100== 0:
            print (i)
        batchP = sess.run(
                pred,
                feed_dict={
                    x_batch: batchX
                })
        a += 1
        b = 'e'+str(a)[::-1]+'s'+(' ')*(9-len(str(a))-2)
        q = ctable.decode(batchP[0])
        if b!=q:
            print ('Failure at ',b,q)
            q = b[::-1]
        else:
            q = q[::-1]
        batchX = np.reshape(ctable.encode(q, MAXLEN),(1,MAXLEN,C))
    
    print ('SUCCESS!!!!!')
    
    '''
    w = [''.join(ctable.indices_char[i] for i in batchY[j]) for j in range(x_train.shape[0])]
    q = [ctable.decode(batchP[i]) for i in range(x_train.shape[0])]
    correct_examples = 0
    for i,j in zip(w,q):
        if (i==j):
            correct_examples += 1
        else :
            print (i,j)
    
    print ('Validation accuracy', float(correct_examples)/x_train.shape[0])
    '''
    
    
    

INFO:tensorflow:Restoring parameters from ./my-model


INFO:tensorflow:Restoring parameters from ./my-model


0
100
('Failure at ', 'e002s    ', 'e001s    ')
200
300
400
500
600
700
800
('Failure at ', 'e009s    ', 'e008s    ')
900
('Failure at ', 'e0001s   ', 'e000s    ')
SUCCESS!!!!!


In [ ]:
print (tf.reverse([[1,2,3]],-1))